In [ ]:
%load_ext autoreload
%autoreload 2

# Examples of CORDEX-CMIP6 cmorization

This notebook should show some examples of how a cmorized CORDEX-CMIP6 dataset could look like.

In [ ]:
!ls ../Tables

## Controlled Vocabulary

The current `CORDEX-CMIP6_CV.json` file contains the controlled vocabulary used by the CMOR3 library for rewriting the model output. Let's have a look:

In [ ]:
import json
import pprint

from IPython.display import JSON

# Opening JSON file
with open("../Tables/CORDEX_CV.json") as json_file:
    CV = json.load(json_file)["CV"]

# JSON(CV['required_global_attributes'])
pprint.pprint(CV["required_global_attributes"])

## Cmorization example

We use the `pyremo.cmor` module to create an example of a CORDEX-CMIP6 dataset.

In [ ]:
import os

import cordex as cx
import xarray as xr
from cordex import cmor as cmor

table_dir = "../Tables"

cmor.set_options(table_prefix="CORDEX")


def test_cmorizer_fx():
    ds = cx.cordex_domain("EUR-11", dummy="topo")
    filename = cmor.cmorize_variable(
        ds,
        "orog",
        mapping_table={"orog": {"varname": "topo"}},
        cmor_table=os.path.join(table_dir, "CORDEX_fx.json"),
        dataset_table=os.path.join(table_dir, "CORDEX_remo_example.json"),
        grids_table=os.path.join(table_dir, "CORDEX_grids.json"),
        CORDEX_domain="EUR-11",
        time_units=None,
        allow_units_convert=True,
    )
    return filename


def test_cmorizer_mon():
    ds = cx.tutorial.open_dataset("remo_EUR-11_TEMP2_mon")
    filename = cmor.cmorize_variable(
        ds,
        "tas",
        mapping_table={"tas": {"varname": "TEMP2"}},
        cmor_table=os.path.join(table_dir, "CORDEX_mon.json"),
        dataset_table=os.path.join(table_dir, "CORDEX_remo_example.json"),
        grids_table=os.path.join(table_dir, "CORDEX_grids.json"),
        CORDEX_domain="EUR-11",
        time_units=None,
        allow_units_convert=True,
    )
    return filename


def test_cmorizer_subdaily(table):
    ds = cx.tutorial.open_dataset("remo_EUR-11_TEMP2_1hr")
    filename = cmor.cmorize_variable(
        ds,
        "tas",
        mapping_table={"tas": {"varname": "TEMP2"}},
        cmor_table=os.path.join(table_dir, f"CORDEX_{table}.json"),
        dataset_table=os.path.join(table_dir, "CORDEX_remo_example.json"),
        grids_table=os.path.join(table_dir, "CORDEX_grids.json"),
        CORDEX_domain="EUR-11",
        time_units=None,
        allow_units_convert=True,
        allow_resample=True,
    )
    return filename

## Example from `fx` table

In [ ]:
f = test_cmorizer_fx()

Let's have a look at the filename.

In [ ]:
f

In [ ]:
!ncdump -h $f

The xarray dataset representation allows to explore the dataset interactively.

In [ ]:
ds = xr.open_dataset(f)
ds

In [ ]:
ds.orog.plot()

In [ ]:
!cdo verifygrid $f

In [ ]:
!cfchecks $f

In [ ]:
!compliance-checker --test=cf:1.7 $f

## Example from `mon` table

In [ ]:
f = test_cmorizer_mon()
f

In [ ]:
ds = xr.open_dataset(f)
ds

In [ ]:
ds.cf

In [ ]:
ds.tas.plot(col="time", col_wrap=4)

In [ ]:
!ncdump -h $f

In [ ]:
!cfchecks $f

In [ ]:
!compliance-checker --test=cf:1.7 $f

## Example from daily and subdaily tables

In [ ]:
f = test_cmorizer_subdaily("1hr")
f

In [ ]:
ds = xr.open_dataset(f)
ds

In [ ]:
!ncdump -h $f

In [ ]:
!cfchecks $f

In [ ]:
!compliance-checker --test=cf:1.7 $f

In [ ]:
# cordex.cmor will automatically resample hourly to other frequencies
# depending on the frequency in the cmor table and the frequency in the
# input dataset
f = test_cmorizer_subdaily("day")
f

In [ ]:
ds = xr.open_dataset(f)
ds

In [ ]:
ds.tas.plot(col="time")

In [ ]:
!ncdump -h $f

In [ ]:
!cfchecks $f

In [ ]:
!compliance-checker --test=cf:1.7 $f